In [1]:
%load_ext autoreload
%autoreload 2

import os
from tqdm.notebook import tqdm, trange

import torch
from torch import nn

from timm.utils import accuracy, AverageMeter

from models import build_model
from data import build_loader
from utils.lr_scheduler import build_scheduler
from utils.optimizer import build_optimizer

In [2]:
def train(model, data_loader, criterion, optimizer):
    model.train()

    loss_meter = AverageMeter()
    acc_meter  = AverageMeter()

    pbar = tqdm(train_loader, leave=False)
    for i, (images, labels) in enumerate( pbar ):
        optimizer.zero_grad()

        images, labels = images.cuda(), labels.cuda()

        output = model(images)
        
        loss = criterion(output, labels)
        acc = accuracy(output, labels, topk=(1,))
        
        loss.backward()
        optimizer.step()

        loss_meter.update(loss.item(), labels.size(0))
        acc_meter.update(acc[0].item(), labels.size(0))

        pbar.set_description(f"[Train] loss: {loss_meter.val:3.3f}, acc: {acc_meter.val:3.3f}")

    pbar.close()
    return loss_meter.avg, acc_meter.avg

def test(model, data_loader):
    model.eval()
    
    loss_meter = AverageMeter()
    acc_meter  = AverageMeter()
    
    pbar = tqdm(test_loader, leave=False)
    with torch.no_grad():
        for i, (images, labels) in enumerate( pbar ):
            images, labels = images.cuda(), labels.cuda()

            output = model(images)
            
            loss = criterion(output, labels)
            acc = accuracy(output, labels, topk=(1,))

            loss_meter.update(loss.item(), labels.size(0))
            acc_meter.update(acc[0].item(), labels.size(0))
            
            pbar.set_description(f"[Test] loss: {loss_meter.val:3.3f}, acc: {acc_meter.val:3.3f}")
    return loss_meter.avg, acc_meter.avg

In [3]:
epoch_num = 2

model = build_model(None)
train_dataset, train_loader, test_dataset, test_loader = build_loader()
lr_scheduler = build_scheduler()
optimizer = build_optimizer(model)
criterion = nn.CrossEntropyLoss().cuda()

model.cuda()
for epoch in range(1, epoch_num+1):
    loss_train, acc_train = train(model, train_loader, criterion, optimizer)
    loss_test, acc_test   = test(model, test_loader)
    print(f"Epoch:{epoch:3} - [train/test] Loss: {loss_train:.2f}/{loss_test:.2f}, Acc: {acc_train:.2f}/{acc_test:.2f}", flush=True)

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch:  1 - [train/test] Loss: 0.71/0.11, Acc: 75.22/96.48


Epoch:  2 - [train/test] Loss: 0.09/0.06, Acc: 97.33/98.05
